### IMPORTS

In [1]:
# load, preprocess, scale, baseline
from wrangle import final_dataset

# manipulate data
import numpy as np

# visualize data
import matplotlib.pyplot as plt
import seaborn as sn

# neural network weights
from modelsNN import good_weights

# optmization problem
from modelsRO import good_problem

np.random.seed(123)

### Load, Clean, Preprocess, Scale, Baseline: Cardiovascular Disease

In [2]:
X_train_scaled_cd, X_test_scaled_cd, y_train_cd, y_test_cd = final_dataset(dataset='cvd')

CVD Loading and Cleaning...
CVD Loaded and Cleaned...

CVD Splitting...
CVD Split...

CVD Scaling...
CVD Scaled...



### Modeling: Cardiovascular Disease

In [3]:
%%time
nn = good_weights(X_train_scaled_cd, y_train_cd,
                  X_test_scaled_cd, y_test_cd)

Running All Algorithms
Running Random Hill Climbing
Running Simulated Annealing
Running Genetic Algorithm
Running Gradient Descent
Compmleted All Algorithms
Time (Seconds): 8.106231689453125e-05
Time (Minutes): 1.3510386149088542e-06 

Working on: Random Hill Climbing
Fitting: Random Hill Climbing
Model Fitting Complete. Time: 301.22599720954895 seconds

Predicting (TRAINING): Random Hill Climbing

Calculating Recall (TRAINING): Random Hill Climbing

Appending Results (TRAINING): Random Hill Climbing

Recall Score (TRAINING): 0.692687514695509%

Predicting (TESTING): Random Hill Climbing

Calculting Recall (TESTING): Random Hill Climbing

Appending Results (TESTING): Random Hill Climbing

Recall Score (TESTING): 0.6897470039946738%


Working on: Simulated Annealing
Fitting: Simulated Annealing
Model Fitting Complete. Time: 2.6548125743865967 seconds

Predicting (TRAINING): Simulated Annealing

Calculating Recall (TRAINING): Simulated Annealing

Appending Results (TRAINING): Simulated A

In [4]:
nn['Random Hill Climbing']

{'nn': NeuralNetwork(clip_max=1, curve=True, hidden_nodes=[4], max_iters=300,
               random_state=123, restarts=200),
 'y_train_pred': array([[1],
        [1],
        [1],
        ...,
        [0],
        [1],
        [1]]),
 'recall_train': 0.692687514695509,
 'y_test_pred': array([[0],
        [0],
        [0],
        ...,
        [0],
        [1],
        [1]]),
 'recall_test': 0.6897470039946738}

### Optimization Problem: Cardiovascular Disease

In [5]:
%%time
opt_prob = good_problem()

Running All Algorithms

Running Random Hill Climbing
Running Random Hill Climbing FourPeaks
Completed Random Hill Climbing FourPeaks. Time: 1.17766006787618 minutes.
Running Random Hill Climbing FlipFlop
Completed Random Hill Climbing FlipFlop. Time: 1.1906298955281576 minutes.
Running Random Hill Climbing Knapsack
Completed Random Hill Climbing Knapsack. Time: 1.2334710041681924 minutes.
Completed Random Hill Climbing. Time: 3.60176096757253

Running Random Hill Climbing
Running Random Hill Climbing FourPeaks
Completed Random Hill Climbing FourPeaks. Time: 1.2140265425046286 minutes.
Running Random Hill Climbing FlipFlop
Completed Random Hill Climbing FlipFlop. Time: 1.2316927750905355 minutes.
Running Random Hill Climbing Knapsack
Completed Random Hill Climbing Knapsack. Time: 1.2373160203297933 minutes.
Completed Random Hill Climbing. Time: 3.6830353379249576

Running Random Hill Climbing
Running Random Hill Climbing FourPeaks
Completed Random Hill Climbing FourPeaks. Time: 1.244896

In [8]:
opt_prob['Random Hill Climbing']['rhc_combo1']['results']

({'df_run_stats_FP':      Iteration  Fitness  FEvals       Time         State  Restarts  max_iters  \
  0            0      6.0       0   0.000125  [1, 1, 0, 0]        10        512   
  1            1      6.0       1   0.001685  [1, 1, 0, 0]        10        512   
  2            2      6.0       2   0.003249  [1, 1, 0, 0]        10        512   
  3            4      6.0       4   0.004896  [1, 1, 0, 0]        10        512   
  4            8      6.0       8   0.006989  [1, 1, 0, 0]        10        512   
  ..         ...      ...     ...        ...           ...       ...        ...   
  688         32      6.0   15126  44.455323  [1, 1, 0, 0]        30        512   
  689         64      6.0   15158  44.680400  [1, 1, 0, 0]        30        512   
  690        128      6.0   15222  45.022006  [1, 1, 0, 0]        30        512   
  691        256      6.0   15350  45.686977  [1, 1, 0, 0]        30        512   
  692        512      6.0   15596  46.850060  [1, 1, 0, 0]        30

### Plotting

In [7]:
# Create/display fitness curve plot for selected algorithm.
# plt.plot(nn.fitness_curve)
# plt.ylabel('Relative Fitness Found')
# plt.show()